Noisy QAOA

In [13]:
import json
import pennylane as qml
import pennylane.numpy as np

In [14]:
edges = [(0, 1), (1, 2), (2, 0), (2, 3)]
num_wires = 4

# We define the Hamiltonian for you!

ops = [qml.PauliZ(0), qml.PauliZ(1),qml.PauliZ(2), qml.PauliZ(3), qml.PauliZ(0)@qml.PauliZ(1), qml.PauliZ(0)@qml.PauliZ(2),qml.PauliZ(1)@qml.PauliZ(2),qml.PauliZ(2)@qml.PauliZ(3)]
coeffs = [0.5, 0.5, 1.25, -0.25, 0.75, 0.75, 0.75, 0.75]

cost_hamiltonian = qml.Hamiltonian(coeffs, ops)

In [15]:
# Write any helper functions you need here

In [16]:
dev = qml.device('default.mixed', wires = num_wires)

@qml.qnode(dev) 
def qaoa_circuit(params, noise_param):

    """
    Define the noisy QAOA circuit with only CNOT and rotation gates, with Depolarizing noise
    in the target qubit of each CNOT gate.

    Args:
        params(list(list(float))): A list with length equal to the QAOA depth. Each element is a list that contains 
        the two QAOA parameters of each layer.
        noise_param (float): The noise parameter associated with the depolarization gate

    Returns: 
        (np.tensor): A numpy tensor of 1 element corresponding to the expectation value of the cost Hamiltonian
    
    """

    # Define QAOA circuit using RX, RZ, CNOT gates with Depolarizing channels
    @qml.qnode(dev)
    def circuit_layer(gamma, alpha):
        # Apply cost Hamiltonian
        for edge in edges:
            qml.CNOT(wires=[edge[0], edge[1]])
            qml.DepolarizingChannel(noise_param, wires=edge[1])

        for i in range(num_wires):
            qml.RZ(params[i, 0], wires=i)
            qml.RX(params[i, 1], wires=i)

        # Apply mixer Hamiltonian
        for wire in range(num_wires):
            qml.RX(alpha, wires=wire)

        return qml.expval(cost_hamiltonian)

    expectation_values = []

    # Apply QAOA layers
    for layer_params in params:
        gamma, alpha = layer_params
        expectation_values.append(circuit_layer(gamma, alpha))

    # Return the expectation value of the cost Hamiltonian
    return np.mean(expectation_values)

In [17]:
def approximation_ratio(qaoa_depth, noise_param):
    """
    Returns the approximation ratio of the QAOA algorithm for the Minimum Vertex Cover of the given graph
    with depolarizing gates after each native CNOT gate

    Args:
        qaoa_depth (float): The number of cost/mixer layer in the QAOA algorithm used
        noise_param (float): The noise parameter associated with the depolarization gate
    
    Returns: 
        (float): The approximation ratio for the noisy QAOA
    """

    # Optimize QAOA parameters to minimize the expectation value of the cost Hamiltonian
    def objective(params):
        return qaoa_circuit(params.reshape(qaoa_depth, 2), noise_param)

    # Initialize random parameters
    initial_params = np.random.rand(qaoa_depth * 2)

    # Use a classical optimizer to find the optimal parameters
    opt = qml.GradientDescentOptimizer(stepsize=0.01)
    params_optimized, _ = opt.step_and_cost(objective, initial_params, max_iterations=100)

    # Calculate the approximation ratio
    true_minimum = qaoa_circuit(np.zeros((qaoa_depth, 2)), 0)
    estimated_minimum = qaoa_circuit(params_optimized.reshape(qaoa_depth, 2), noise_param)
    approximation_ratio = estimated_minimum / true_minimum

    return approximation_ratio


In [18]:
# These functions are responsible for testing the solution.
random_params = np.array([np.random.rand(2)])

ops_2 = [qml.PauliX(0), qml.PauliX(1), qml.PauliX(2), qml.PauliX(3)]
coeffs_2 = [1,1,1,1]

mixer_hamiltonian = qml.Hamiltonian(coeffs_2, ops_2)

@qml.qnode(dev)
def noiseless_qaoa(params):

    for wire in range(num_wires):

        qml.Hadamard(wires = wire)

    for elem in params:

        qml.ApproxTimeEvolution(cost_hamiltonian, elem[0], 1)
        qml.ApproxTimeEvolution(mixer_hamiltonian, elem[1],1)

    return qml.expval(cost_hamiltonian)

random_params = np.array([np.random.rand(2)])

circuit_check = (np.isclose(noiseless_qaoa(random_params) - qaoa_circuit(random_params,0),0)).numpy()

def run(test_case_input: str) -> str:
    input = json.loads(test_case_input)
    output = approximation_ratio(*input)

    return str(output)

def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    
    tape = qaoa_circuit.qtape
    names = [op.name for op in tape.operations]
    random_params = np.array([np.random.rand(2)])

    assert circuit_check, "qaoa_circuit is not doing what it's expected to."

    assert names.count('ApproxTimeEvolution') == 0, "Your circuit must not use the built-in PennyLane Trotterization."
     
    assert set(names) == {'DepolarizingChannel', 'RX', 'RY', 'RZ', 'CNOT'}, "Your circuit must use qml.RX, qml.RY, qml.RZ, qml.CNOT, and qml.DepolarizingChannel."

    assert solution_output > expected_output - 0.02

# These are the public test cases
test_cases = [
    ('[2,0.005]', '0.4875'),
    ('[1, 0.003]', '0.1307')
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

IndexError: index 1 is out of bounds for axis 0 with size 1